In [15]:
import pandas as pd
import numpy as np
from clases.NeuronalNetwork import SimpleNeuralNetwork
from clases.Optimizer import SGDOptimizer, AdamOptimizer
from clases.Trainer import Trainer
import warnings
from clases.ActivationFunction import *

In [12]:
warnings.filterwarnings("ignore")

data = pd.read_csv('../datos/data.csv')
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

test_size = int(0.1 * m)
data_test = data[:test_size, :].T  # El 10% inicial para el conjunto de prueba

# El conjunto provisional restante (90%)
remaining_data = data[test_size:, :]

train_size = int(0.8 * m)
data_train = remaining_data[:train_size, :].T  # El siguiente 80% para entrenamiento
data_validation = remaining_data[train_size:, :].T  # El resto 10% para validación

X_train, Y_train = data_train[1:], data_train[0]
X_val, Y_val = data_validation[1:], data_validation[0]
X_test, Y_test = data_test[1:], data_test[0]

In [17]:
model = SimpleNeuralNetwork(input_size=784, output_size=10, activation_function=relu, activation_function_d=relu_derivative, output_function=softmax)
optimizer = SGDOptimizer(learning_rate=0.001)
trainer = Trainer(model, optimizer)
trainer.train(X_train, Y_train,X_val,Y_val, epochs=500)

-----------------------------------------------------------
| end of epoch  10 | time:  0.49s | valid accuracy    0.283 
-----------------------------------------------------------
| end of epoch  20 | time:  0.44s | valid accuracy    0.560 
-----------------------------------------------------------
| end of epoch  30 | time:  0.51s | valid accuracy    0.685 
-----------------------------------------------------------
| end of epoch  40 | time:  0.44s | valid accuracy    0.743 
-----------------------------------------------------------
| end of epoch  50 | time:  0.47s | valid accuracy    0.773 
-----------------------------------------------------------
| end of epoch  60 | time:  0.45s | valid accuracy    0.796 
-----------------------------------------------------------
| end of epoch  70 | time:  0.46s | valid accuracy    0.811 
-----------------------------------------------------------
| end of epoch  80 | time:  0.47s | valid accuracy    0.787 
--------------------------------

In [16]:
model = SimpleNeuralNetwork(input_size=784, output_size=10, activation_function=relu, activation_function_d=relu_derivative, output_function=softmax)
optimizer = AdamOptimizer(learning_rate=0.001)
trainer = Trainer(model, optimizer)
trainer.train(X_train, Y_train,X_val,Y_val, epochs=500)

-----------------------------------------------------------
| end of epoch  10 | time:  0.46s | valid accuracy    0.766 
-----------------------------------------------------------
| end of epoch  20 | time:  0.44s | valid accuracy    0.857 
-----------------------------------------------------------
| end of epoch  30 | time:  0.44s | valid accuracy    0.884 
-----------------------------------------------------------
| end of epoch  40 | time:  0.45s | valid accuracy    0.902 
-----------------------------------------------------------
| end of epoch  50 | time:  0.42s | valid accuracy    0.909 
-----------------------------------------------------------
| end of epoch  60 | time:  0.47s | valid accuracy    0.915 
-----------------------------------------------------------
| end of epoch  70 | time:  0.45s | valid accuracy    0.919 
-----------------------------------------------------------
| end of epoch  80 | time:  0.50s | valid accuracy    0.920 
--------------------------------